In [2]:
import pandas as pd
import re
import spacy
import os
import string
from spacy.tokenizer import Tokenizer
from spacy.tokens import Doc
from spacy.util import minibatch, compounding
import nltk
from tqdm import tqdm

In [2]:
files = os.listdir('data/articles/')
files = [file for file in files if file != ".DS_Store"]

In [3]:
text_dat = []
for file in files:
    fname = 'data/articles/' + file
    f = open(fname, encoding='utf8', errors='replace')
    article = f.read()
    # removing nonsense chars
    printable = set(string.printable)
    article = ''.join(filter(lambda x: x in printable, article))
    text_dat.append(article)

f.close()

## Spacification

In [4]:
nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
article1 = nlp(text_dat[1])

In [5]:
for ent in article1.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

AFP 1 4 ORG
Jewel SamadChinese 10 28 PERSON
Alibaba 49 56 ORG
Jack Ma 65 72 PERSON
the New York Stock Exchange 128 155 ORG
New York 159 167 GPE
September 19 168 180 DATE
2014New York 182 194 GPE
AFP 196 199 ORG
2014 218 222 DATE
2000 280 284 DATE
Chinese 328 335 NORP
Alibaba 351 358 PRODUCT
Renaissance Capital 494 513 ORG
this week 531 540 DATE
Russia 580 586 GPE
Ukraine 608 615 GPE
the Middle East 633 648 LOC
US 723 725 GPE
Renaissance 742 753 ORG
Renaissance 760 771 ORG
273 788 791 CARDINAL
2014 808 812 DATE
23 percent 817 827 PERCENT
2013 833 837 DATE
291 883 886 CARDINAL
27 percent 901 911 PERCENT
2013 917 921 DATE
another heady year 951 969 DATE
2015 973 977 DATE
Airbnb 1117 1123 GPE
Uber 1151 1155 PERSON
US 1173 1175 GPE
$85 billion 1198 1209 MONEY
2014 1213 1217 DATE
Renaissance 1232 1243 ORG
about 55 percent 1245 1261 PERCENT
2013 1275 1279 DATE
Bob Greifeld 1408 1420 PERSON
Nasdaq 1422 1428 ORG
CNBC 1454 1458 ORG
The year 2000 1501 1514 DATE
406 1557 1560 CARDINAL
$96 billion 

Peshmerga 19480 19489 GPE
British 19504 19511 NORP
US 19516 19518 GPE
UK 19569 19571 GPE
Mr Hawrami 19676 19686 PERSON
Iraq 19896 19900 GPE
Hawrami 20074 20081 PERSON
Isil 20133 20137 ORG
UK 20155 20157 GPE
Europe 20162 20168 LOC
Britain 20266 20273 GPE
Saudi Arabia's 20423 20437 GPE
US 20539 20541 GPE
the International Monetary Fund 20637 20668 ORG
25 20690 20692 MONEY
Fund 20703 20707 ORG
100-$105reference 20711 20728 MONEY
Saudi 20764 20769 NORP
2018 20818 20822 DATE
Saudis 20832 20838 NORP
2018 20854 20858 DATE
2000 21138 21142 DATE
December 15 years ago 21147 21168 DATE
a few weeks 21190 21201 DATE
this festive season 21271 21290 DATE
San Francisco 21361 21374 GPE
eight 21396 21401 CARDINAL
2013 21467 21471 DATE
fifth 21481 21486 ORDINAL
America 21505 21512 GPE
Janet Yellen 21629 21641 PERSON
the Federal Reserve 21655 21674 ORG
Alan Greenspan 21781 21795 PERSON
1999 21825 21829 DATE
Alibaba 21894 21901 GPE
Chinese 21905 21912 NORP
New York 21943 21951 GPE
September 21955 21964 DAT

Goldman Sachs 58626 58639 ORG
The Wall Street Journal 58705 58728 ORG
Joshua Birnbaum 58730 58745 PERSON
Goldman Sachs Group Inc. 58754 58778 ORG
more than $2 billion 58864 58884 MONEY
Tilden Park Capital Management LP 58927 58960 ORG
Monday 59014 59020 DATE
Journal 59026 59033 ORG
about 15% 59196 59205 PERCENT
A few weeks ago 59231 59246 DATE
Brown 59271 59276 PERSON
Journal 59459 59466 PERSON
this year 59487 59496 DATE
less than 1% 59511 59523 PERCENT
about 7% 59539 59547 PERCENT
last year 59548 59557 DATE
15% 59562 59565 PERCENT
2012 59569 59573 DATE
Friday 59578 59584 DATE
multiyear 59637 59646 DATE
7% 59662 59664 PERCENT
spring 2013 59727 59738 DATE
first 59923 59928 ORDINAL
Treasury 60107 60115 ORG
REUTERS 60233 60240 ORG
Maxim ShemetovRussian 60241 60262 PERSON
Vladimir Putin 60273 60287 PERSON
Evgeny Gavrilenkov 60289 60307 PERSON
Russia 60362 60368 GPE
Sberbank CIB 60386 60398 ORG
Russian 60437 60444 NORP
Russia 60469 60475 GPE
the Central Bank of Russia 60549 60575 ORG
17% 61

1998 86005 86009 DATE
three 86087 86092 CARDINAL
Juckes 86143 86149 PERSON
Socit GnraleRussia 86186 86204 PRODUCT
Brazil 86237 86243 GPE
Korea 86245 86250 GPE
Goldman Sachs' 86444 86458 ORG
David Kostin 86459 86471 PERSON
2014 86473 86477 DATE
this year 86670 86679 DATE
Kostin 86682 86688 PERSON
Friday 86698 86704 DATE
Only 12% 86707 86715 PERCENT
500 86777 86780 CARDINAL
1997 86809 86813 DATE
7% 86829 86831 PERCENT
500 86840 86843 CARDINAL
October 86856 86863 DATE
11% 86894 86897 PERCENT
two months 86919 86929 DATE
the past weeks 86958 86972 DATE
Goldman Sachs FREE AppDownload 87085 87115 ORG
CNBCBill Gross 87116 87130 PERSON
Jeff Gundlach 87132 87145 PERSON
CNBC 87195 87199 ORG
Monday 87203 87209 DATE
Janus Capital's 87211 87226 ORG
Bill Gross 87227 87237 PERSON
TIPS 87248 87252 ORG
Treasury 87257 87265 ORG
Treasury 87470 87478 ORG
last week 87639 87648 DATE
Jeff Gundlach 87671 87684 PERSON
Gundlach 87709 87717 PERSON
TIPS 87766 87770 ORG
Earlier this month 88086 88104 DATE
Bill Gros

In [6]:
articles_processed = [nlp(article) for article in text_dat]

## Percentages

- using built in model to predict and find percentage

In [ ]:
def return_percentages(article_spacified):
    pcts = [ent.text for ent in article_spacified.ents if ent.label_ == "PERCENT"]
    return pcts

In [ ]:
pct_lst = []
for article in articles_processed:
    pcts = return_percentages(article)
    pct_lst.extend(pcts)

In [ ]:
# just distinct occurences
pct_uniq = list(set(pct_lst))

In [ ]:
pct_df = pd.DataFrame({'percentages' : pct_uniq})

In [ ]:
pct_df.to_csv("results/percentage_matches.csv")

## Labeling Data

Spacy takes json as input.
<br>
1. Train-test split, 75 ish articles, because spacy uses recurrent neural net that does CV, and test error calc
    - do not want to overfit and want to extract CEOs not found in the train data
2. For each label (company and ceo): 
    - iterate over data to find position of matches
    - compile to json structure based on article. 
3. Train spacy model. 

In [ ]:
# import random
# random.seed(308)
# train_text = random.sample(text_dat, 75)

In [3]:
ceos = pd.read_csv("data/labels/ceo.csv", header=None, encoding="ISO-8859-1")
companies = pd.read_csv("data/labels/companies.csv", header=None)
# some ceos have two spaces between f name l name
ceos[2] = [' '.join(s.split()) for s in ceos[2]]
ceo_lst = list(set(ceos[2]))

In [4]:
company_lst = list(set(companies[0]))
len(company_lst)

2592

In [9]:
[ceo for ceo in ceo_lst if ceo in company_lst]

['Paul Singer',
 'Pershing Square',
 'Steve Blumenthal',
 'Swanson',
 'Third Point',
 'Garland',
 'American Apparel',
 'Office Depot',
 'Laurent Potdevin',
 'Murray Kessler',
 'Interpublic Group']

In [10]:
# a few weird cases in our labeled data
not_ceos = ['American Apparel','Interpublic Group', 'Office Depot',
            'Pershing Square', 'Third Point', 'Swanson']
not_companies = ['Murray Kessler', 'Steve Blumenthal', 'Garland', 
                 'Paul Singer', 'Laurent Potdevin']
ceo_lst = [ceo for ceo in ceo_lst if ceo not in not_ceos]
company_lst = [company for company in company_lst if company not in not_companies]

In [ ]:
# # function that takes article, list of candidate labels, and the entity name
# def posn_finder(article, labels, entity_name):
#     result=[]
#     article_labels = [x for x in labels if x in article]
#     if len(article_labels) == 0:
#         # if empty return empty list
#         return article_labels
#     else: 
#         for label in article_labels:
#             entities = [(m.start(), m.end(), entity_name) for m in re.finditer(label, article)]
#             result.extend(entities)
            
#     return result

In [ ]:
# iterating over all articles for both ceos and companies. 
# then extending list by index

In [ ]:
# article_ceos = []
# for article in train_text:
#     ceo_posns = posn_finder(article, ceo_lst, "CEO")
#     article_ceos.append(ceo_posns)

In [ ]:
# article_companies = []
# for article in train_text:
#     company_posns = posn_finder(article, company_lst, "Company")
#     article_companies.append(company_posns)

In [ ]:
# labeled_data = []
# for i in range(len(article_ceos)):
#     labeled_data.append(article_ceos[i] + article_companies[i])

In [ ]:
# spacy_data = [(train_text[i], {"entities": labeled_data[i]}) for i in range(len(train_text))]

## Model Construction

Training data is ready, so building spacy model ***from scratch***, like Grandmama. 

Running the recurrent neural network on 75 articles. Since neural net uses SGD and also iterates through data, should be able to make good predictions that do not overfit to data. Spacy recommends having a few hundred labeled observations. 75 should be sufficient to meet this criterion.

In [ ]:
# nlp = spacy.blank('en')
# ner = nlp.create_pipe("ner")
# nlp.add_pipe(ner)
# iterations = 125

In [ ]:
# ner.add_label("CEO")
# ner.add_label("Company")


In [ ]:
# optimizer = nlp.begin_training()
# sizes = compounding(1.0, 4.0, 1.001)

In [ ]:
# ## overlapping entity labels ended this effort. 
# for itn in range(iterations):
#     random.shuffle(spacy_data)
#     batches = minibatch(spacy_data, size=sizes)
#     losses = {}
#     for batch in batches:
#         texts, annotations = zip(*batch)
#         nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        
#         print("Losses", losses)

## Take 9000

1. Find sentences with ORGs or Person with Spacy
2. Extract features from the sentences.
3. Prediction time!
    - train test split
    - model selection
    - ...
4. Pull out sentences predicted to have CEO or Company
    - use spacy to pull out the person or org within the sentence.



In [13]:
def filter_sentences(space_article, entity):
    sentences = [sent.text for sent in space_article.sents]
    entities = [ent.text for ent in space_article.ents if ent.label_ == entity]
    sent_w_ent = []
    for s in sentences:
        if any(ent in s for ent in entities):
            sent_w_ent.append(s)
    
    return sent_w_ent

In [14]:
## getting all sentences with organizations
org_sentences = []
for article in tqdm(articles_processed):
    sent_w_org = filter_sentences(article, "ORG")
    org_sentences.extend(sent_w_org)

100%|██████████| 730/730 [01:01<00:00, 11.95it/s]


In [15]:
## getting all sentences with CEOs
person_sentences = []
for article in tqdm(articles_processed):
    sent_w_person = filter_sentences(article, "PERSON")
    person_sentences.extend(sent_w_person)

100%|██████████| 730/730 [00:48<00:00, 15.06it/s]


In [16]:
print(len(org_sentences))
print(len(person_sentences))

303707
199827


In [17]:
org_df = pd.DataFrame({'sentences': org_sentences})
person_df = pd.DataFrame({'sentences': person_sentences})

In [18]:
def match(sentence, labels=None):
    if any(lbl in sentence for lbl in labels):
        return 1
    else:
        return 0

In [19]:
org_df['y'] = org_df.sentences.apply(lambda x: match(x, labels=company_lst))

In [20]:
person_df['y'] = person_df.sentences.apply(lambda x: match(x, labels=ceo_lst))

In [23]:
org_df.describe()

,y
count,303707.000000
mean,0.380752
std,0.485573
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [24]:
person_df.describe()

,y
count,199827.000000
mean,0.127650
std,0.333701
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [38]:
def count_caps(sent):
    return sum([1 for s in sent if s.isupper()])

In [39]:
org_df['num_cap'] = org_df.sentences.apply(lambda x: count_caps(x))

In [40]:
person_df['num_cap'] = person_df.sentences.apply(lambda x: count_caps(x))

In [41]:
def num_words(sent):
    tokes = sent.split(" ")
    return len(tokes)

In [42]:
org_df['num_words'] = org_df.sentences.apply(lambda x: num_words(x))
person_df['num_words'] = person_df.sentences.apply(lambda x: num_words(x))

In [50]:
org_df.head()

,sentences,y,num_cap,num_words,has_quote
0,(Reuters) - The Federal Reserve's vow to keep ...,1,6,45,1
1,I think 'considerable time' captures about as ...,0,12,46,1
2,Atlanta Federal Reserve Bank President Dennis ...,1,8,14,1
3,"In October, the Fed restated the pledge, but a...",0,3,32,1
4,JPMorgan economist Michael Feroli said in a re...,1,6,51,1


In [51]:
person_df.head()

,sentences,y,num_cap,num_words,has_quote
0,I think 'considerable time' captures about as ...,0,12,46,1
1,Atlanta Federal Reserve Bank President Dennis ...,1,8,14,1
2,JPMorgan economist Michael Feroli said in a re...,0,6,51,1
3,Jan Hatzius of Goldman Sachs has said it would...,1,10,35,1
4,(Reporting by Michael Flaherty in Washington ...,1,22,34,1


In [48]:
def contains_quote(text):
    if '\'' in text or '\"' in text:
        return 1
    else:
        return 0

In [49]:
person_df['has_quote'] = person_df.sentences.apply(lambda x: contains_quote(x))
org_df['has_quote'] = org_df.sentences.apply(lambda x: contains_quote(x))

In [63]:
pd.crosstab(person_df.has_quote, person_df.y)

y,0,1
has_quote,,
0,93486,12212
1,80833,13296


In [64]:
pd.crosstab(org_df.has_quote, org_df.y)

y,0,1
has_quote,,
0,118696,61143
1,69374,54494


In [80]:
# using an article from 2013 about highly rated ceos
best_ceos = ['Mark Zuckerberg', 'Zuckerberg', 'Bill McDermott', 'McDermott', 'Jim Hagemann Snabe',
                      'Snabe', 'Jim Snabe', 'Dominic Barton', 'Barton', 'Jim Turley', 'Turley', 'John Schlifske',
                     'Schlifske', 'Jeff Bezos', 'Bezos', 'Page', 'Larry Page', 'Musk', 'Elon Musk']
# same thing but for companies
best_companies = ['Google, Inc.', 'Google', 'SAS', 'Facebook', 
                  'Guidewire', 'Workday', 'Intel', 'YouTube',
                 'Amazon', 'Tesla']

In [81]:
org_df['best_company'] = org_df.sentences.apply(lambda x: match(x, best_companies))
person_df['best_ceo'] = person_df.sentences.apply(lambda x: match(x, best_ceos))

In [82]:
pd.crosstab(org_df.best_company, org_df.y)

y,0,1
best_company,,
0,188063,109376
1,7,6261


pd.crosstab(person_df.best_ceo, person_df.y)

In [87]:
org_df['contain_company'] = org_df.sentences.apply(lambda x: match(x, ['company','Company']))
person_df['contain_ceo'] = person_df.sentences.apply(lambda x: 
                                                     match(x, ['CEO', 'Chief executive', 'chief executive']))

These are all the features I will personally extract manually

### Writing to CSV

In [89]:
org_df.to_csv('data/labeled_org_data.csv', index=False)
person_df.to_csv('data/labeled_person_data.csv', index=False)